In [ ]:
#feature importance

In [2]:
from datetime import datetime
from datetime import date

from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
K.image_data_format()

from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

import time
import io
import os
import re
import pandas as pd
import numpy as np

In [3]:
seriea_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/seriea_{current_date}.csv'.format(current_date=date.today()))
ligue1_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/ligue1_{current_date}.csv'.format(current_date=date.today()))
laliga_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/laliga_{current_date}.csv'.format(current_date=date.today()))
bun_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/bun_{current_date}.csv'.format(current_date=date.today()))
epl_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/epl_{current_date}.csv'.format(current_date=date.today()))



# merging top 5 combined stats

top5_raw = seriea_raw.append(ligue1_raw)
top5_raw = top5_raw.append(laliga_raw)
top5_raw = top5_raw.append(bun_raw)
top5_raw = top5_raw.append(epl_raw)


"""
# using 20-21 stats for early season projections

top5_raw = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_2020-21.csv')

# list of promoted teams to drop

prom = ['Empoli','Salernitana','Venezia','Espanyol','Mallorca','RayoVallecano','Bochum','GreutherFurth','Troyes','Clermont','NorwichCity','Brentford','Watford']
"""

#top5_raw = top5_raw.drop('Unnamed: 0', axis = 1)
top5_raw['psxg'] = top5_raw['psxg'] / top5_raw['gp']
top5_raw['npxg'] = top5_raw['npxg'] / top5_raw['gp']
top5_raw['np:G-xG'] = top5_raw['np:G-xG'] / top5_raw['gp']
top5_raw['Prog'] = top5_raw['Prog'] / top5_raw['gp']
top5_stats = top5_raw.drop('gp', axis = 1)
top5_stats

top5_stats_home = top5_stats
top5_stats_home = top5_stats_home.rename(columns = {'Squad':'Home'})
top5_stats_home = top5_stats_home[['Home', 'npxg','npxG/Sh','np:G-xG','Prog','SCA90','GCA90','Poss','psxg']]
top5_stats_home

top5_stats_away = top5_stats
top5_stats_away = top5_stats_away.rename(columns = {'Squad':'Away'})
top5_stats_away = top5_stats_away[['Away', 'npxg','npxG/Sh','np:G-xG','Prog','SCA90','GCA90','Poss','psxg']]
top5_stats_away

# switch to dataset with promoted teams after ~ 4 weeks

top5_sched = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_sched_21-22.csv')
# top5_sched = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_sched_21-22_noprom.csv')
top5_sched = top5_sched[['Wk','Date','Home','Away']]
top5_sched

top5_matchup = pd.merge(top5_sched, top5_stats_home, on = 'Home', how = "left")
top5_matchup = pd.merge(top5_matchup, top5_stats_away, on = 'Away', how = "left")

#pd.DataFrame(totals_pred).to_csv('/Users/matthewfalcona/FalconaForecast/totals_pred_top5.csv')

#pd.DataFrame(top5_matchup).to_csv('/Users/matthewfalcona/FalconaForecast/matchup_test.csv')


########################################################################################

# creating train and test sets for results model

top5_hist = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_hist.csv')

top5_result_tt = top5_hist.drop(['Wk','Date','Home','Away','Total','ou'], axis = 1)
top5_result_tt.head()
top5_result_tt.shape

(7203, 17)

In [5]:
y = top5_result_tt.Result
x = top5_result_tt.drop('Result', axis = 1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

x_train.shape


(5762, 16)

In [6]:
x_test.shape


(1441, 16)

In [7]:
y_train.shape


(5762,)

In [8]:
y_test.shape

(1441,)

In [11]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(y_train)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_test)
y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(y_test)

y_train.shape

(5762, 3)

In [12]:
def base_model():
    model = Sequential() # for training and inference features
    model.add(Dense(50, input_dim = 16, kernel_initializer='normal', activation='tanh'))
    model.add(Dropout(.1))
    model.add(Dense(3, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return(model)


# model fitting

start = time.time()  # TRACK TIME

model = base_model()

model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50, batch_size = 100, verbose = 1)

scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

# # MODEL - RESULTS


end = time.time()
final_time = end-start
print(final_time)

test_acc = model.predict(x_test)

Epoch 1/50
58/58 [==============================] - 1s 12ms/step - loss: 1.0311 - accuracy: 0.4756 - val_loss: 1.0049 - val_accuracy: 0.5094
Epoch 2/50
58/58 [==============================] - 0s 1ms/step - loss: 1.0030 - accuracy: 0.5134 - val_loss: 1.0012 - val_accuracy: 0.5003
Epoch 3/50
58/58 [==============================] - 0s 1ms/step - loss: 0.9921 - accuracy: 0.5077 - val_loss: 0.9922 - val_accuracy: 0.5170
Epoch 4/50
58/58 [==============================] - 0s 1ms/step - loss: 0.9807 - accuracy: 0.5225 - val_loss: 0.9878 - val_accuracy: 0.5198
Epoch 5/50
58/58 [==============================] - 0s 1ms/step - loss: 0.9817 - accuracy: 0.5179 - val_loss: 0.9914 - val_accuracy: 0.5115
Epoch 6/50
58/58 [==============================] - 0s 1ms/step - loss: 0.9853 - accuracy: 0.5213 - val_loss: 0.9838 - val_accuracy: 0.5226
Epoch 7/50
58/58 [==============================] - 0s 1ms/step - loss: 0.9844 - accuracy: 0.5257 - val_loss: 0.9974 - val_accuracy: 0.5024
Epoch 8/50
58/58 [=

In [13]:
top5_pred = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_sched_21-22.csv')
# top5_pred = pd.read_csv('/Users/matthewfalcona/FalconaForecast/datasets/top5_sched_21-22_noprom.csv')
# top5_pred = top5_pred[~top5_pred.isin(prom)]
top5_pred = top5_pred[['Home','Away']]
top5_pred = pd.merge(top5_pred, top5_stats_home, on = 'Home', how = "left")
top5_pred = pd.merge(top5_pred, top5_stats_away, on = 'Away', how = "left")
# top5_pred = top5_pred.dropna()
top5_pred

top5_pred_model = top5_pred.drop(columns = ['Home','Away'])

probs = model.predict(top5_pred_model)


In [ ]:
pd.DataFrame(probs).to_csv('/Users/matthewfalcona/FalconaForecast/probs_top5.csv')
pd.DataFrame(top5_pred).to_csv('/Users/matthewfalcona/FalconaForecast/top5_pred.csv')

In [16]:
perm = PermutationImportance(model, random_state=1, scoring="average_precision").fit(x_test, y_test)
results_feat_imp = eli5.show_weights(perm, feature_names = x_test.columns.tolist())
results_feat_imp

Weight,Feature
0.0193 ± 0.0104,GCA90_y
0.0158 ± 0.0056,psxg_y
0.0152 ± 0.0115,SCA90_y
0.0120 ± 0.0081,psxg_x
0.0052 ± 0.0032,Prog_x
0.0033 ± 0.0068,SCA90_x
0.0033 ± 0.0040,Poss_y
0.0027 ± 0.0034,np:G-xG_y
0.0023 ± 0.0039,npxg_y
0.0017 ± 0.0081,GCA90_x


In [19]:
from sklearn.inspection import permutation_importance


# perform permutation importance
results = permutation_importance(model, x, y, scoring='average_precision')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

ValueError: multiclass format is not supported

In [18]:
from sklearn.metrics import classification_report
print(classification_report(x_test, y_test))

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multilabel-indicator targets